In [39]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier

train_data=pd.read_csv("/content/train.csv")
test_data=pd.read_csv("/content/test.csv")
# women survival
women_survival=train_data.loc[train_data.Sex=="female"]["Survived"]
women_survival_rate=sum(women_survival)/len(women_survival)
print(f"Percentage of women survival : {round(women_survival_rate*100,2)}")
# men survival
men_survival=train_data.loc[train_data.Sex=="male"]["Survived"]
men_survival_rate=sum(men_survival)/len(men_survival)
print(f"Percentage of women survival : {round(men_survival_rate*100,2)}")

# prediction model

Y=train_data["Survived"]
features=["Pclass", "Sex", "Age", "SibSp", "Parch"]
X=pd.get_dummies(train_data[features])
X.fillna(X.median(), inplace=True)
x_test=pd.get_dummies(test_data[features])
x_test.fillna(x_test.median(),inplace=True)
model=RandomForestClassifier(n_estimators=100,max_depth=5,random_state=1)
model.fit(X,Y)
predict=model.predict(x_test)
output=pd.DataFrame({'PassengerId':test_data.PassengerId,'Survival':predict})
output.to_csv('/content/sample_data/survival_prediction.csv')
print("submission successful")

Percentage of women survival : 74.2
Percentage of women survival : 18.89
submission successful


In [40]:
import pickle
pickle_out=open("Titanic_model.pkl","wb")
pickle.dump(model,pickle_out)
pickle_out.close()

In [41]:
!pip install streamlit


In [59]:
%%writefile app.py
from pyngrok import ngrok
import streamlit as st
import pickle
import numpy as np
from PIL import Image

# Load the model
def load_model():
    with open("Titanic_model.pkl", "rb") as model_file:
        return pickle.load(model_file)

model = load_model()

# Titanic Theme Colors
st.set_page_config(page_title="Titanic Survival Prediction", page_icon="🌊", layout="centered")
st.markdown(
    """
    <style>
    body {
        background-color: #001f3f;  /* Dark navy blue */
        color: #FFD700;  /* Gold */
    }
    .stButton > button {
        background: linear-gradient(to right, #FFD700, #FFA500);
        color: black;
        font-size: 18px;
        border-radius: 12px;
    }
    .stTextInput > div > input {
        background-color: #ffffff;
        color: black;
    }
    .stSelectbox > div {
        background-color: #ffffff;
        color: black;
    }
    </style>
    """,
    unsafe_allow_html=True,
)

# Titanic Header
st.image("https://upload.wikimedia.org/wikipedia/commons/f/fd/RMS_Titanic_3.jpg", use_column_width=True)
st.markdown("# 🌊 Titanic Survival Prediction")
st.write("### Enter passenger details to check survival probability")

# Form for Input
with st.form(key="titanic_form"):
    pclass = st.selectbox("Passenger Class", ["1st Class", "2nd Class", "3rd Class"], index=2)
    sex = st.selectbox("Sex", ["Male", "Female"], index=0)
    age = st.slider("Age", min_value=0, max_value=100, value=30, step=1)
    sibsp = st.selectbox("Siblings/Spouses Aboard", list(range(11)), index=0)
    parch = st.selectbox("Parents/Children Aboard", list(range(11)), index=0)
    embarked = st.selectbox("Port of Embarkation", ["C", "Q", "S"], index=2)  # C = Cherbourg, Q = Queenstown, S = Southampton
    submit = st.form_submit_button("Predict Survival")

# Prediction Logic
if submit:
    pclass_encoded = {"1st Class": 1, "2nd Class": 2, "3rd Class": 3}[pclass]
    sex_encoded = 1 if sex == "Female" else 0  # Encoding: Female = 1, Male = 0
    embarked_encoded = {"C": 0, "Q": 1, "S": 2}[embarked]  # Encoding ports
    features = np.array([[pclass_encoded, sex_encoded, age, sibsp, parch, embarked_encoded]])
    prediction = model.predict(features)[0]
    probability = model.predict_proba(features)[0][1] * 100  # Probability of survival

    if prediction == 1:
        st.success(f"Survival Probability: {probability:.2f}% - You are likely to survive! 🌟")
    else:
        st.error(f"Survival Probability: {probability:.2f}% - Survival is uncertain. 💦")

# Ngrok Setup
ngrok.set_auth_token("2v5IArlnsiADJTXKU1Qs7kA5oJ0_3CzEvuzTFE7wvNKu1919J")
public_url = ngrok.connect(addr="8501", proto="http")
print("Streamlit App URL:", public_url)

Overwriting app.py


In [43]:
!pip install pyngrok


In [44]:
!pip install pyngrok streamlit


In [60]:
from pyngrok import ngrok
ngrok.set_auth_token("2v5IArlnsiADJTXKU1Qs7kA5oJ0_3CzEvuzTFE7wvNKu1919J")  # Replace with your actual ngrok auth token
public_url = ngrok.connect(addr="8501", proto="http")
print("Streamlit App URL:", public_url)


Streamlit App URL: NgrokTunnel: "https://b042-35-185-46-197.ngrok-free.app" -> "http://localhost:8501"


In [20]:
!streamlit run app.py &>/dev/null &


In [ ]:
def predict_survival(pclass, sex, age, sibsp, parch):
    # Create input dataframe
    input_data = pd.DataFrame({
        "Pclass": [pclass],
        "Sex": [sex],
        "Age": [age],
        "SibSp": [sibsp],
        "Parch": [parch]
    })

    input_data = pd.get_dummies(input_data)
    input_data = input_data.reindex(columns=X.columns, fill_value=0)

    for col in X.columns:
        if col not in input_data:
            input_data[col] = 0

    prediction = model.predict(input_data)

    return "Survived ✅" if prediction[0] == 1 else "Did Not Survive ❌"


In [ ]:
    pclass = int(input("Enter Passenger Class (1, 2, 3): "))
    sex = input("Enter Sex (male/female): ").lower()
    age = float(input("Enter Age: "))
    sibsp = int(input("Enter Number of Siblings/Spouses: "))
    parch = int(input("Enter Number of Parents/Children: "))
    print(predict_survival(pclass, sex, age, sibsp, parch))

Enter Passenger Class (1, 2, 3): 3
Enter Sex (male/female): female
Enter Age: 55
Enter Number of Siblings/Spouses: 0
Enter Number of Parents/Children: 0
Survived ✅
